Function definition (to be removed, just 1 in the all code)

In [1]:
import requests
import pandas as pd
import geopandas as gpd
import json
from shapely import wkt
import ipywidgets as widgets
import matplotlib.pyplot as plt
import folium
from shapely.geometry import mapping

def read_response(t):
    try:
        data = t.json() #This will convert the response to a json object
        return data
    except requests.exceptions.JSONDecodeError:
        print("Invalid response JSON!")
        print("Response content::", t.text)
        data = None 

def get_measurement_unit(pollutant):
    t=requests.post(url="http://127.0.0.1:5000/api/units", json={"var_pollutant": pollutant}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data[0]
    

## DV_10
The interface allow the user to select a pollutant and visualize a topographic map, created with contour lines of 100m difference, where for each pollutant is shown the monthly average registered and the station location.  
The different layers selection makes possible to choose for what year(s) to visualize the stations data.



In [ ]:
t=requests.get(url="http://127.0.0.1:5000/api/pollutant")
list_pollutant = read_response(t)
Selected_pollutant = widgets.Dropdown(
    options=list_pollutant,
    value=list_pollutant[0],
    default=list_pollutant[0],
    description='Pollutant:',
    disabled=False, #Because we want the user to be able to select the province
)
Selected_pollutant

In [ ]:
# select the data we want as parameters
data = {"var_pollutant": Selected_pollutant.value } #data is a dictionary

# send the request to the server, we use post because we are sending data to the server
t=requests.post(url="http://127.0.0.1:5000/api/DV_10", json=data) #json= data will convert the dictionary to a json object and send it to the server

data = read_response(t) 


Adjustments to the geodataframe

In [ ]:
# to visualize the response, we can conver the data to a pandas geodataframe
gdf = gpd.GeoDataFrame(data)
gdf['geometry'] = gdf['geometry'].apply(wkt.loads)
gdf.set_geometry('geometry', inplace=True)

gdf = gdf.dropna(subset=['quota']) # Drop rows where 'quota' is NaN

gdf['month'] = pd.to_datetime(gdf['month'], errors='coerce')
gdf['month'] = gdf['month'].dt.strftime('%b %Y')

print(gdf)

In [ ]:
# upload GeoDataFrame with contour lines
gdf_contour = gpd.read_file("../DATA/contour_lines.geojson") #this will be changed with "contour_lines.geojson"
gdf_contour

In [ ]:
import branca.colormap as cm
import folium

# Colormap for elevation and monthly average
colormap_el = cm.LinearColormap(["green", "yellow", "brown"], vmin=gdf_contour['elevation'].min(), vmax=gdf_contour['elevation'].max())
colormap_c = cm.LinearColormap(["blue", "yellow", "red"], vmin=gdf['monthly_average'].min(), vmax=gdf['monthly_average'].max())
 
# Style function for contour lines
def style_function(feature):
    return {
        "color": colormap_el(feature["properties"]["elevation"]),
        "weight": 0.25
    }

# Get years from gdf
years = sorted(gdf["month"].str[-4:].unique())  

m = folium.Map(location=[45.4642, 9.300], zoom_start=8, tiles='CartoDB positron')  
              
folium.GeoJson(gdf_contour, name="Contour lines", style_function=style_function).add_to(m)

unit = get_measurement_unit(Selected_pollutant.value)
for year in years:
    if year == years[-2]:  
        layer = folium.FeatureGroup(name=f"{year}",show=True)
    else:
        layer = folium.FeatureGroup(name=f"{year}",show=False)
    gdf_filtered = gdf[gdf["month"].str.contains(str(year))]
 
    for _, row in gdf_filtered.iterrows():
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            tooltip=f"{row['month']}: {row['monthly_average']}{unit}",
            radius=5,
            color=colormap_c(row["monthly_average"]),
        ).add_to(layer)
    layer.add_to(m) 

colormap_el.caption = "Elevation (m)"
colormap_el.add_to(m)
colormap_c.caption = f"Monthly average {Selected_pollutant.value} ({unit})"
colormap_c.add_to(m)
folium.LayerControl(collapsed=False).add_to(m)

# Add title to the map
title_html = '''
    <h3 align="center" style="font-size:20px"><b>Correlation between concentration and elevation map</b></h3>
'''

m.get_root().html.add_child(folium.Element(title_html))
m

## DV_11
The interface allow the user to select a pollutant and visualize a graph that
highlights the correlation between a monthly average of the selected pollutant and the altitude of the sensors.  
It is possible to select both the year and the month for the visualization.


In [2]:
t=requests.get(url="http://127.0.0.1:5000/api/pollutant")
list_pollutant = read_response(t)
Selected_pollutant = widgets.Dropdown(
    options=list_pollutant,
    value=list_pollutant[0],
    default=list_pollutant[0],
    description='Pollutant:',
    disabled=False, #Because we want the user to be able to select the province
) 
Selected_pollutant

Dropdown(description='Pollutant:', options=('Ammoniaca', 'Arsenico', 'Benzene', 'Benzo(a)pirene', 'Biossido di…

In [3]:
# select the data we want as parameters
data = {"var_pollutant": Selected_pollutant.value } #data is a dictionary

# send the request to the server, we use post because we are sending data to the server
t=requests.post(url="http://127.0.0.1:5000/api/DV_10", json=data) #json= data will convert the dictionary to a json object and send it to the server

data = read_response(t)

In [4]:
# to visualize the response, we can conver the data to a pandas dataframe
df = gpd.GeoDataFrame(data)

df['month'] = pd.to_datetime(df['month'], errors='coerce')
df['month'] = df['month'].dt.strftime('%b %Y')

print(df)

                            geometry     month  monthly_average  \
0    POINT (10.04384767 45.14254358)  Jan 2018         4.213997   
1    POINT (10.00620528 45.27848997)  Jan 2018        26.472140   
2     POINT (9.66625758 45.23349689)  Jan 2018        23.742857   
3     POINT (9.16464919 45.19468231)  Jan 2018         3.061186   
4     POINT (8.90418742 45.10277464)  Jan 2018         8.256873   
..                               ...       ...              ...   
750   POINT (8.90418742 45.10277464)  Jan 2025         7.100000   
751       POINT (9.231667 45.478347)  Jan 2025        11.150000   
752    POINT (9.38468725 46.1381409)  Jan 2025        11.200000   
753     POINT (9.64366225 45.691044)  Jan 2025        12.300000   
754  POINT (11.07609728 45.01688067)  Jan 2025        10.900000   

                                         nome_stazione  quota  
0                         Cremona Via Fatebenefratelli   43.0  
1    Corte de' Cortesi con Cignone v. Villa Strada ...   57.0  
2  

In [6]:
import ipywidgets as widgets
import matplotlib.pyplot as plt

# Get measurement unit for the selected pollutant
unit = get_measurement_unit(Selected_pollutant.value)

# Get unique years
years = sorted(df["month"].str[-4:].unique())

# Create year slider
year_slider = widgets.IntSlider(
    value=int(years[0]),
    min=int(years[0]),
    max=int(years[-1]),
    step=1,
    description="Year:"
)

# Function to update available months based on selected year
def get_months(year):
    months = sorted(df[df["month"].str.contains(str(year))]["month"].str[:3].unique(),
                     key=lambda m: ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"].index(m))
    return ["All"] + months  # Include "All" option

# Create month dropdown (default: all months)
month_dropdown = widgets.Dropdown(
    options=get_months(year_slider.value),
    value="All",
    description="Month:"
)

# Update month dropdown when year changes
def update_month_options(change):
    month_dropdown.options = get_months(change["new"])

year_slider.observe(update_month_options, names="value")

# Function to update plot
def update_plot(year, month):
    df_filtered = df[df["month"].str.contains(str(year))]

    # Separate selected month from others
    if month != "All":
        df_selected = df_filtered[df_filtered["month"].str.startswith(month)]
        df_other = df_filtered[~df_filtered["month"].str.startswith(month)]
    else:
        df_selected = df_filtered
        df_other = pd.DataFrame()  # Empty DataFrame
    
    fig, ax = plt.subplots(figsize=(6, 5))
        
    # Plot selected month with full opacity
    ax.scatter(df_selected['monthly_average'], df_selected['quota'], c=df_selected['monthly_average'], cmap='coolwarm', edgecolors='k', marker='o', alpha=1.0)
    # Plot other months with transparency
    if not df_other.empty:
        ax.scatter(df_other['monthly_average'], df_other['quota'], c=df_other['monthly_average'], cmap='coolwarm', marker='o', alpha=0.2)
    
    plt.title(f"Correlation between monthly average of {Selected_pollutant.value} and station height")
    plt.xlabel(f"Monthly Average per pollutant {unit}")
    plt.ylabel("Height [m]")
    ax.grid(True)

    plt.show()

# Connect sliders to update function
out = widgets.interactive_output(update_plot, {"year": year_slider, "month": month_dropdown})

# Display sliders and plot
display(year_slider, month_dropdown, out)


IntSlider(value=2018, description='Year:', max=2025, min=2018)

Dropdown(description='Month:', options=('All', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', …

Output()